# Experiment 1

This notebook contains the experiment to **generate model projections in an uncontrolled scenario until April 12**, which is reported on Figure 2 in the [paper](https://arxiv.org/abs/2004.07641).

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [ ]:
import numpy as np
import pickle
import multiprocessing

In [ ]:
from lib.mobilitysim import MobilitySimulator
from lib.parallel import launch_parallel_simulations
from lib.distributions import CovidDistributions
from lib.data import collect_data_from_df
from lib.measures import (MeasureList, Interval, 
                          SocialDistancingForPositiveMeasure)

#### Set the random seed for reproducibility

In [ ]:
# Choose random seed
c = 0
# Set it
np.random.seed(c)
# Define prefix string used to save plots
runstr = f'run{c}_'  

#### Set the number of roll-outs to simulate

In [ ]:
random_repeats = 40 # Set to at least 20 to obtain stable results

#### Define the number of parallel workers used for simulations 

In [ ]:
num_workers = multiprocessing.cpu_count() - 1

#### Set the time to simulate

In [ ]:
days_until_lockdown = 13 # March 10 - March 23
days_present = 35  # Days since March 10 on April 12 (March 10 is time 0 for our experiments)
max_time_present = 24.0 * (days_present) # in hours

#### Define population/mobility parameters

Import downsampled version of Tübingen (population downsampled by 20x, sites by 10x) that was used used to generate mobility traces on the fly for inference (sites, home locations, etc)

In [ ]:
case_downsample = 10                       # Downsample of cases
mob_settings = 'lib/tu_settings_20_10.pk'  # Mobility settings (should match `case_downsample`)
town_name = 'LK Tübingen'                  # Name of town to fetch case data

# See town-generator.ipynb for an example on how to create the settings
with open(mob_settings, 'rb') as fp:
    obj = pickle.load(fp)
mob = MobilitySimulator(**obj)

#### Load the case data

We adjust the number of cases to the down-scaling of the population

In [ ]:
new_cases_ = collect_data_from_df(town_name, 'new', until=days_present)
resistant_cases_ = collect_data_from_df(town_name, 'recovered', until=days_present)
fatality_cases_ = collect_data_from_df(town_name, 'fatality', until=days_present)

# fatality rate per age group
num_age_groups = fatality_cases_.shape[1] 
fatality_rates_by_age = (fatality_cases_[-1, :] / \
    (new_cases_[-1, :] + fatality_cases_[-1, :] + resistant_cases_[-1, :]))
print('Empirical fatality rates per age group:  ', fatality_rates_by_age.tolist())

# Instantiate correct state transition distributions (estimated from in literature)
distributions = CovidDistributions(fatality_rates_by_age=fatality_rates_by_age)

# Scale down cases based on number of people in simulation
new_cases, resistant_cases, fatality_cases = (
    1/case_downsample * new_cases_, 
    1/case_downsample * resistant_cases_, 
    1/case_downsample * fatality_cases_)
new_cases, resistant_cases, fatality_cases = np.ceil(new_cases), np.ceil(resistant_cases), np.ceil(fatality_cases)

#### Define initial seed count per state 

These numbers are based on the *downscaled* infection counts on March 10.

In [ ]:
initial_seeds = {
    'expo' : 8,
    'ipre' : 1,
    'isym' : 3,
    'iasy' : 3,
}

#### Set epidemic parameters

Inferred using Bayesian optimization.

In [ ]:
beta = 0.91
inferred_params = {
    'betas' : {'education': beta,'social': beta,'bus_stop': beta,'office': beta,'supermarket': beta}, # site infectivity by type
    'mu': 1.0
}

#### Define standard testing parameters

The parameters are the same used for inference.

In [ ]:
def standard_testing(max_time):
    standard_testing_params = {
        'testing_t_window'    : [0.0, max_time], # in hours
        'testing_frequency'   : 24.0,     # in hours
        'test_reporting_lag'  : 48.0,     # in hours (actual and self-report delay)
        'tests_per_batch'     : 10,       # assume 300 tests/day in LK Tübingen
        'test_fpr'            : 0.0,      # test false positive rate
        'test_fnr'            : 0.0,      # test false negative rate
        'test_smart_delta'    : 24.0 * 3, # in hours
        'test_smart_duration' : 24.0 * 7, # in hours
        'test_smart_action'   : 'isolate', 
        'test_smart_num_contacts'   : 10, 
        'test_targets'        : 'isym',
        'test_queue_policy'   : 'fifo',
        'smart_tracing'       : None, 
    }
    return standard_testing_params

#### Define helper I/O functions to load/save the output of the experiment

In [ ]:
def save_summary(summary, filename):
    with open('summaries/' + filename, 'wb') as fp:
        pickle.dump(summary, fp)
    
def load_summary(filename):
    with open('summaries/' + filename, 'rb') as fp:
        summary = pickle.load(fp)
    return summary

## 2. Run the simulations

Define a helper function to run general type of experiment, fixing all parameters above.

In [ ]:
def run(tparam, measure_list, t, local_seeds, dynamic_tracing=False):

    # add standard measure of positives staying isolated
    measure_list +=  [
        SocialDistancingForPositiveMeasure(
            t_window=Interval(0.0, t), p_stay_home=1.0)
    ]
    measure_list = MeasureList(measure_list)

    # run simulations
    summary = launch_parallel_simulations(
        mob_settings, 
        distributions, 
        random_repeats, multiprocessing.cpu_count(), 
        inferred_params, local_seeds, tparam, measure_list, 
        max_time=t, 
        num_people=mob.num_people, 
        num_sites=mob.num_sites, 
        site_loc=mob.site_loc, 
        home_loc=mob.home_loc, 
        dynamic_tracing=dynamic_tracing,
        verbose=False)
    return summary
    

Run the experiment using the parameters defined above and simulate until April 12 with a single measure, symptomatic individuals staying home.

**WARNING: this cell might take a long time to run depending of the parameters defined above!**

In [ ]:
testing_params_opt = standard_testing(max_time_present)
measure_list = []

summary_INF = run(testing_params_opt, measure_list, max_time_present, initial_seeds)

save_summary(summary_INF, 'summary_INF.pk')

## 3. Plot the results

#### Import libs

In [ ]:
from lib.plot import Plotter

Load the summary holding the results of the experiment to plot.

In [ ]:
summary_INF = load_summary('summary_INF.pk')

Reproduce Figure 2 (a) in the paper.

In [ ]:
plotter = Plotter()
plotter.plot_positives_vs_target(
    summary_INF, new_cases.sum(axis=1), 
    test_lag=2,
    title=('Predictions for uncontrolled scenario after March 23'
           'for inferred parameters'), 
    filename=runstr + 'opt_00',
    figsize=(6,4),
    start_date='2020-03-10',
    errorevery=1, acc=500, 
    lockdown_at=days_until_lockdown,
    ymax=250)

Reproduce Figure 2 (b) in the paper.

In [ ]:
targets = new_cases.sum(axis=1)
targets = np.hstack((targets[0], np.diff(targets)))

plotter = Plotter()
plotter.plot_daily_infected(
    summary_INF,
    title='', 
    filename=runstr + 'opt_001',
    figsize=(6, 4),
    legend_loc=0,
    errorevery=100, acc=1000, 
    lockdown_at=days_until_lockdown,
    lockdown_label_y=500,
    start_date='2020-03-10',
    show_target=targets,
    ymax=1800)

Reproduce Figure 2 (c) in the paper.

In [ ]:
plotter = Plotter()
plotter.plot_cumulative_infected(
    summary_INF,
    title='',
    filename=runstr + 'opt_002',
    figsize=(6, 4),
    legend_loc='upper left',
    errorevery=100, acc=1000, 
    start_date='2020-03-10',
    lockdown_at=days_until_lockdown,
    lockdown_label_y=500,
    show_target=targets,
    ymax=None)

Reproduce Figure 2 (d) in the paper.

In [ ]:
plotter = Plotter()
plotter.plot_daily_rts(
    summary_INF,
    filename=runstr + 'opt_003',
    start_date='2020-03-10',
    sigma=None, # prior found by MLE, set to None to recompute
    figsize=(6, 4), 
    subplot_adjust=None,
    lockdown_label='Lockdown', 
    lockdown_at=days_until_lockdown,
    lockdown_label_y=2.6,
    titles=None,
    ci=0.6,
    ymax=4.0)